## 1 Laplacian Embeddings, PCA and SVD

This exercise guides you through relating Laplacian Embeddings, Principal Component Analysis (PCA), and Singular Value Decomposition (SVD) through pseudoinverses and incidence matrices.

#### Task a

An inverse of a real matrix does not always exists. A generalization of the matrix inverse is known as the pseudoinverse,also known as the Moore-Penrose inverse. One way to calculate the inverse of a real matrix M is to calculate its Singular Value Decomposition (SVD) $M = U \Sigma V^T$ before taking its inverse $M^+ = V \Sigma^+ U^T$ , where the pseudoinverse $\Sigma^+ = \Sigma^{-1}$ of the diagonal singular value matrix $\Sigma$ calculated by simply taking the reciprocal of each its elements. Recall that the eigendecomposition of a real symmetric matrix L can be written as:

$$
\begin{align*}
L = Q \Delta Q^T = M M ^ T
\end{align*}
$$

where $M = Q \sqrt \Delta$. Show that $L = U \Sigma^T \Sigma U^T$. This iss one way eigendecomposition and SVD are related for real symmetric matrices.

Lets rewrite the formula:

$$
\begin{align*}
L &= M M^T = \\
&= U \Sigma V^T  (U \Sigma V^T) ^ T\\
&= U \Sigma V^T V \Sigma^T U ^ T\\
&= U \Sigma (V^T V) \Sigma^T U ^ T
\end{align*}
$$

Considering that fact that

$V$ is the orthogonal matrix so the $V^T$ is also orthogonal matrix. And their multiplication is equal to 1-matrix. So, $V^T V = 1$.

$\Sigma$ is the diagonal matrix so $\Sigma = \Sigma^T$

So at the end of the day:

$$
\begin{align*}
M M^T &= U \Sigma (V^T V) \Sigma^T U ^ T\\
&= U \Sigma \Sigma^T U ^ T\\
&= U \Sigma^T \Sigma U ^ T
\end{align*}
$$

## Task b

We can define $L^+ = (M^T)^+ M^+ = X^TX$ where $X = M^+$. This creates a centered "data matrix" X so $L^+$ can be interpreted as a covariance matrix. Now consider when L is the Laplacian of a simple graph G. Argue how the objective of Principal Component Analysis (PCA) of X is equivalent to that of spectral embedding of G via eigendecomposition of its Laplacian L(Laplacian Embedding).

First let`s consider a PCA objective for the X. The $L^+ = X^TX$ is a covariance matrix so the objective is:

$$
\begin{align*}
\max_{\vec{u_1}} \vec{u_1} ^T L^+ \vec{u_1} &= \max_{\vec{u_1}} \vec{u_1} ^T X^TX \vec{u_1} \\
&= \max_{\vec{u_1}} \vec{u_1}^T (M^+)^T M^+ \vec{u_1} \\
&= \max_{\vec{u_1}} \vec{u_1}^T (V \Sigma^+ U^T)^T V \Sigma^+ U^T \vec{u_1} \\
&= \max_{\vec{u_1}} U (\Sigma^+)^T V^T V \Sigma^+ U^T \vec{u_1} \\
&= \max_{\vec{u_1}} U (\Sigma^+)^T \Sigma^+ U^T \vec{u_1} \\
&= \max_{\vec{u_1}} U (\Sigma^{-1})^T \Sigma^{-1} U^T \vec{u_1}
\end{align*}
$$

At the ame time the objective function of the laplacian embeddings for the L Laplacian matrix:

$$
\begin{align*}
\min_{\vec{u_2}} \vec{u_2}^T L \vec{u_2} &= \min_{\vec{u_2}} \vec{u_2}^T U \Sigma^T \Sigma U^T \vec{u_2}
\end{align*}
$$


So, in the PCA case the solution $\vec{u_1}$ will be the first d eigenvectors of the corressponding d largest eigenvalues. At th same time the eigenvalues of the laplacian matrix will be just inversed eigenvalues of the $L^+$ with the same eigenvectorss whoch also will be the olution in thiss case because we take d eigenvectors which correspond to d lowest eigenvalues in this time. So, both solutions are the same here. 

#### Task c

Consider the following simple graph.

![graph](1_c.png)

Write down its unnormalized Laplacian L.

In [1]:
import scipy as sp
import numpy as np
import pathpy as pp
import pathpyG as ppg
from scipy.linalg import svd, inv, eig
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding

In [2]:
network = ppg.Graph.from_edge_list([
    ("1", "2"),
    ("1", "3"),
    ("2", "3"),
    ("3", "4"),
    ("4", "5"),
    ("4", "6"),
    ("5", "6")
]).to_undirected()
ppg.plot(network)

In [3]:
L = network.get_laplacian().todense()
L

matrix([[ 2., -1., -1.,  0.,  0.,  0.],
        [-1.,  2., -1.,  0.,  0.,  0.],
        [-1., -1.,  3., -1.,  0.,  0.],
        [ 0.,  0., -1.,  3., -1., -1.],
        [ 0.,  0.,  0., -1.,  2., -1.],
        [ 0.,  0.,  0., -1., -1.,  2.]], dtype=float32)

#### Task d

Calculate its “data matrix” X and show that PCA of X is equivalent to that of its spectral embedding via eigendecomposition of its Laplacian L (Laplacian Embedding)

So, first we will find $Q$ and $\sqrt{\Lambda}$

In [4]:
ew, ev = eig(L)
Q = ev[:,np.argsort(-ew)]
ew = -np.sort(-ew)
Lambda = np.diag(ew)

Now $M$, SVD of it and then $M^+$, so finally X

In [ ]:
M = Q @ np.sqrt(np.abs(Lambda))

In [ ]:
V, Sigma, U_T = svd(M, full_matrices=True)

In [11]:
X = (U_T.T @ np.diag(1 / Sigma) @ V.T)

In [12]:
pca_embedding = PCA(n_components=3).fit_transform(X)
pca_embedding

array([[ 7.9765765e+02, -3.0008525e-01,  5.0199269e-03],
       [ 7.9765723e+02, -3.2437798e-01, -1.2492170e-02],
       [ 7.9765723e+02, -3.0209878e-01, -6.0632247e-01],
       [ 7.9765723e+02, -2.9833099e-01,  6.1553311e-01],
       [ 7.9765729e+02,  1.2246594e+00, -1.7004479e-03],
       [-3.9882864e+03, -4.6712586e-05,  7.5817024e-06]], dtype=float32)

In [13]:
def laplacian_embedding(network, d=None, normalization = None):
    """Function that returns a vector representation of all nodes based on the entries of eigenvectors of the laplacian"""
    
    if d is None:
        d = network.N - 1
    ew, ev = sp.linalg.eig(network.get_laplacian(normalization=normalization).todense())

    # indices of non-zero eigenvalues
    nonzero_indices = np.where(~np.isclose(ew, 0, atol=1e-5))[0]

    # select eigenvectors and eigenvalues corresponding to non-zero eigenvalues
    ev = ev[:, nonzero_indices]
    ew = ew[nonzero_indices]

    # Sort eigenvalues and eigenvectors according to eigenvalue magnitude
    sort_indices = np.argsort(ew)
    ew = ew[sort_indices]
    ev = ev[:, sort_indices]

    return ev[:, :d].real

In [15]:
l_emb = laplacian_embedding(network, d=3)
l_emb

array([[ 0.46470514,  0.02630652,  0.0131867 ],
       [ 0.4647051 , -0.55195224, -0.27667776],
       [ 0.26095632,  0.52564585,  0.26349115],
       [-0.2609565 ,  0.5256457 ,  0.263491  ],
       [-0.46470514, -0.30697098, -0.7439981 ],
       [-0.46470514, -0.21867466,  0.48050714]], dtype=float32)

#### Task e

Instead of constructing the Laplacian as L = D−A where D and A are the degree and adjacency matrices, respectively, the Laplacian can be constructed via incidence matrices. The incidence matrix B describes the connectivity between nodes and edges. For the simple graph above the incidence matrix can be written as:

![one_more_graph](1_e.png)


where its rows represent nodes and its columns represent edges. Note how every edge is incident to only 2 nodes, and the incidence matrix requires an orientation, labelling one incidence 1 and another −1. The choice of which incidence is labelled 1 or −1 is arbitrary, and is simply a matter of bookkeeping 1. For the simple graph above, calculate L = BB T .


In [16]:
B = np.array([
    [1,  1,  0,  0,  0,  0,  0],
    [-1, 0,  1,  0,  0,  0,  0],
    [0,  -1, -1, 1,  0,  0,  0],
    [0,  0,  0,  -1, 1,  1,  0],
    [0,  0,  0,  0,  -1, 0,  1],
    [0,  0,  0,  0,  0,  -1, -1]
])
B

array([[ 1,  1,  0,  0,  0,  0,  0],
       [-1,  0,  1,  0,  0,  0,  0],
       [ 0, -1, -1,  1,  0,  0,  0],
       [ 0,  0,  0, -1,  1,  1,  0],
       [ 0,  0,  0,  0, -1,  0,  1],
       [ 0,  0,  0,  0,  0, -1, -1]])

In [17]:
L2 = B @ B.T
L2

array([[ 2, -1, -1,  0,  0,  0],
       [-1,  2, -1,  0,  0,  0],
       [-1, -1,  3, -1,  0,  0],
       [ 0,  0, -1,  3, -1, -1],
       [ 0,  0,  0, -1,  2, -1],
       [ 0,  0,  0, -1, -1,  2]])

In [18]:
L

matrix([[ 2., -1., -1.,  0.,  0.,  0.],
        [-1.,  2., -1.,  0.,  0.,  0.],
        [-1., -1.,  3., -1.,  0.,  0.],
        [ 0.,  0., -1.,  3., -1., -1.],
        [ 0.,  0.,  0., -1.,  2., -1.],
        [ 0.,  0.,  0., -1., -1.,  2.]], dtype=float32)

#### Task f pretty same as g

$L = BB^T$ holds more generally than for this example. Show that (generally, for simple graphs) the singular vectors of Singular Value Decompostion (SVD) of B are equivalent to the eigenvectors of eigendecomposition of L.

In [37]:
u, s, v_t = svd(B, full_matrices=True)

In [38]:
u.round(4)

array([[ 0.1845,  0.    , -0.    , -0.7638,  0.4647, -0.4082],
       [ 0.1845, -0.0602, -0.5311,  0.5455,  0.4647, -0.4082],
       [-0.6572,  0.0602,  0.5311,  0.2182,  0.261 , -0.4082],
       [ 0.6572,  0.0602,  0.5311,  0.2182, -0.261 , -0.4082],
       [-0.1845, -0.7327, -0.1859, -0.1091, -0.4647, -0.4082],
       [-0.1845,  0.6725, -0.3452, -0.1091, -0.4647, -0.4082]])

In [39]:
ew, ev = eig(L)

In [40]:
ev.round(4)

array([[ 0.1845, -0.7638, -0.4082,  0.4647,  0.0263,  0.0132],
       [ 0.1845,  0.5455, -0.4082,  0.4647, -0.552 , -0.2767],
       [-0.6572,  0.2182, -0.4082,  0.261 ,  0.5256,  0.2635],
       [ 0.6572,  0.2182, -0.4082, -0.261 ,  0.5256,  0.2635],
       [-0.1845, -0.1091, -0.4082, -0.4647, -0.307 , -0.744 ],
       [-0.1845, -0.1091, -0.4082, -0.4647, -0.2187,  0.4805]],
      dtype=float32)